<a href="https://colab.research.google.com/github/nivetharaja26/Google_Colab/blob/main/bert_imdb_sentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets tensorflow


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from datasets import load_dataset

# 1. Load dataset
dataset = load_dataset("imdb")

# Split data
train_texts = dataset["train"]["text"][:5000]   # taking smaller subset for faster training
train_labels = dataset["train"]["label"][:5000]

test_texts = dataset["test"]["text"][:1000]
test_labels = dataset["test"]["label"][:1000]

# 2. Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

max_len = 128

def tokenize(texts):
    return tokenizer(
        texts,
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_tensors="tf"
    )

train_encodings = tokenize(train_texts)
test_encodings = tokenize(test_texts)

# Convert to tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
)).batch(32)

# 3. Load BERT base model
bert = TFBertModel.from_pretrained("bert-base-uncased", use_safetensors=False)

# 4. Build classification model
input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

outputs = bert(input_ids, attention_mask=attention_mask)
cls_output = outputs.last_hidden_state[:, 0, :]  # [CLS] token representation

x = tf.keras.layers.Dropout(0.3)(cls_output)
out = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=out)

# 5. Compile
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# 6. Train
history = model.fit(train_dataset, validation_data=test_dataset, epochs=2)

# 7. Evaluate
loss, acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {acc:.2f}")

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

tf_model.h5:   0%|          | 0.00/536M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training

Epoch 1/2


/usr/local/lib/python3.12/dist-packages/tf_keras/src/engine/functional.py:641: UserWarning: Input dict contained keys ['token_type_ids'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


157/157 [==============================] - 9775s 62s/step - loss: 0.0115 - accuracy: 0.9942 - val_loss: 1.4309e-05 - val_accuracy: 1.0000
Epoch 2/2
32/32 [==============================] - 637s 20s/step - loss: 8.6253e-06 - accuracy: 1.0000
Test Accuracy: 1.00
